In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!git init
!git remote add origin https://github.com/AlexOlsen/DeepWeeds
!git pull origin master

In [ ]:
import datetime,time,os
import pandas as pd
import numpy as np
import csv
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger
from keras.optimizers import Adam
from keras.models import Model, load_model
from sklearn.metrics import confusion_matrix, classification_report
from keras.applications.inception_v3 import InceptionV3
from keras.applications import VGG19
from keras.utils import to_categorical
from keras.layers import Dense, GlobalAveragePooling2D
import keras
from keras.applications.inception_v3 import InceptionV3

import gc
from sklearn.model_selection import train_test_split

In [ ]:
OUTPUT_DIRECTORY="./output/"
# CONSTANTS 
LABEL_DIRECTORY = "./labels/"
MODEL_DIRECTORY = "./models/"
IMG_DIRECTORY = "/kaggle/input/images"
RAW_IMG_SIZE = (224, 224)
IMG_SIZE = (224, 224)
INPUT_SHAPE = (IMG_SIZE[0], IMG_SIZE[1], 3)
MAX_EPOCH = 100
BATCH_SIZE = 32
FOLDS = 5
STOPPING_PATIENCE = 4
LR_PATIENCE = 16
INITIAL_LR = 0.0001
CLASSES = [0, 1, 2, 3, 4, 5, 6, 7, 8]
CLASS_NAMES = ['Chinee Apple',
               'Lantana',
               'Parkinsonia',
               'Parthenium',
               'Prickly Acacia',
               'Rubber Vine',
               'Siam Weed',
               'Snake Weed',
               'Negatives']



output_directory = OUTPUT_DIRECTORY
if not os.path.exists(output_directory):
        os.makedirs(output_directory)

In [ ]:
df=pd.read_csv(LABEL_DIRECTORY+"labels.csv")
least_freq=sorted(df.Species.value_counts())[0]
temp=pd.DataFrame()
for species in  set(df["Species"]):
    temp = pd.concat([temp,df[df["Species"]==species].sample(least_freq,random_state=49)])
temp.to_csv("./resampled.csv",index=False)

df=pd.read_csv("./resampled.csv")

In [ ]:
df=pd.read_csv("./resampled.csv")
one_hot=pd.get_dummies(df['Species'])
df=df.join(one_hot)
df=df.drop(columns=['Label','Species'],axis=1)
df.columns


In [ ]:
train_dataframe,test_dataframe=train_test_split(df,random_state=49,shuffle=True,train_size=0.75)
val_dataframe,test_dataframe=train_test_split(test_dataframe,random_state=49,shuffle=True,test_size=0.5)

In [ ]:
# train_dataframe['Label']=to_categorical(train_dataframe['Label']).tolist()
# val_dataframe['Label']  =to_categorical(val_dataframe['Label']).tolist()
# test_dataframe['Label'] =to_categorical(test_dataframe['Label']).tolist()

train_image_count = train_dataframe.shape[0]
val_image_count = train_dataframe.shape[0]
test_image_count = test_dataframe.shape[0]

In [ ]:
y_colums=['Chinee apple', 'Lantana', 'Negative', 'Parkinsonia','Parthenium', 'Prickly acacia', 'Rubber vine', 'Siam weed','Snake weed']
train_data_generator = ImageDataGenerator(
    rescale=1. / 255,
    fill_mode="constant",
    shear_range=0.2,
    zoom_range=(0.5, 1),
    horizontal_flip=True,
    rotation_range=360,
    channel_shift_range=25,
    brightness_range=(0.75, 1.25))
# Validation image augmentation
val_data_generator = ImageDataGenerator(
    rescale=1. / 255,
    fill_mode="constant",
    shear_range=0.2,
    zoom_range=(0.5, 1),
    horizontal_flip=True,
    rotation_range=360,
    channel_shift_range=25,
    brightness_range=(0.75, 1.25))
# No testing image augmentation (except for converting pixel values to floats)
test_data_generator = ImageDataGenerator(rescale=1. / 255)
train_data_generator = train_data_generator.flow_from_dataframe(
    train_dataframe,
    IMG_DIRECTORY,
    x_col='Filename',
    y_col=y_colums,
    target_size=RAW_IMG_SIZE,
    batch_size=BATCH_SIZE,
    has_ext=True,
    class_mode='raw')
# Load validation images in batches from directory and apply rescaling
val_data_generator = val_data_generator.flow_from_dataframe(
    val_dataframe,
    IMG_DIRECTORY,
    x_col="Filename",
    y_col=y_colums,
    target_size=RAW_IMG_SIZE,
    batch_size=BATCH_SIZE,
    has_ext=True,
    class_mode='raw')
# Load test images in batches from directory and apply rescaling
test_data_generator = test_data_generator.flow_from_dataframe(
    test_dataframe,
    IMG_DIRECTORY,
    x_col="Filename",
    y_col=y_colums,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    has_ext=True,
    shuffle=False,
    class_mode='raw')

In [ ]:
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)
base_model = VGG19(weights='imagenet', include_top=False, input_shape=INPUT_SHAPE)

x = base_model.output
# Add a global average pooling layer
x = GlobalAveragePooling2D(name='avg_pool')(x)
# Add fully connected output layer with sigmoid activation for multi label classification
outputs = Dense(len(CLASSES), activation='softmax', name='fc9')(x)
# Assemble the modified model
model = Model(inputs=base_model.input, outputs=outputs)

In [ ]:
# Checkpoints for training
model_checkpoint = ModelCheckpoint(output_directory + "lastbest-0.keras", verbose=1, save_best_only=True,monitor='val_loss')
early_stopping = EarlyStopping(monitor="val_loss",min_delta=0,patience=STOPPING_PATIENCE,verbose=2,mode="auto",restore_best_weights=True,start_from_epoch=10)
tensorboard = TensorBoard(log_dir=output_directory, histogram_freq=0, write_graph=True, write_images=False)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.5, patience=LR_PATIENCE, min_lr=0.000003125)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=INITIAL_LR),
              loss="categorical_crossentropy",
              metrics=['categorical_accuracy'])

checkpoint_callback = ModelCheckpoint(filepath=output_directory+'model_checkpoint.keras',
                                      monitor='val_loss',
                                      verbose=1,
                                      save_best_only=True,
                                      mode='min')
# csv_logger = CSVLogger(output_directory + "training_metrics.csv")
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

global_epoch = 0
restarts = 0
last_best_losses = []
last_best_epochs = []
# Train model until MAX_EPOCH, restarting after each early stop when learning has plateaued

# Define callbacks

while global_epoch < MAX_EPOCH:
    history = model.fit(
        train_data_generator,
        epochs=MAX_EPOCH - global_epoch,
        validation_data=val_data_generator,
        callbacks=[tensorboard, model_checkpoint, early_stopping, reduce_lr],
        shuffle=False)
    last_best_losses.append(min(history.history['val_loss']))
    last_best_local_epoch = history.history['val_loss'].index(min(history.history['val_loss']))
    last_best_epochs.append(global_epoch + last_best_local_epoch)

    if early_stopping.stopped_epoch == 0:
        print("Completed training after {} epochs.".format(MAX_EPOCH))
        break
    else:
        global_epoch = global_epoch + early_stopping.stopped_epoch - STOPPING_PATIENCE + 1
        print("Early stopping triggered after local epoch {} (global epoch {}).".format(
            early_stopping.stopped_epoch, global_epoch))
        print("Restarting from last best val_loss at local epoch {} (global epoch {}).".format(
            early_stopping.stopped_epoch - STOPPING_PATIENCE, global_epoch - STOPPING_PATIENCE))
        restarts = restarts + 1
        model.compile(loss= "categorical_crossentropy", optimizer=Adam(learning_rate=INITIAL_LR / 2 ** restarts),
                      metrics=['categorical_accuracy'])
        model_checkpoint = ModelCheckpoint(output_directory + "lastbest-{}.keras".format(restarts),
                                           monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        gc.collect()

In [ ]:
# Save last best model info
with open(output_directory + "last_best_models.csv", 'w', newline='') as file:
     writer = csv.writer(file, delimiter=',')
     writer.writerow(['Model file', 'Global epoch', 'Validation loss'])
     for i in range(restarts + 1):
         writer.writerow(["lastbest-{}.keras".format(i), last_best_epochs[i], last_best_losses[i]])
    # Load the last best model
model = load_model(output_directory + "lastbest-{}.keras".format(last_best_losses.index(min(last_best_losses))))

In [ ]:
# Evaluate model on test subset for kth fold
predictions = model.predict(test_data_generator)
y_true=np.argmax(test_data_generator.labels,axis=1)
y_pred=np.argmax(predictions, axis=1)
# y_pred[np.max(predictions, axis=1) < 1 / 9]


In [ ]:
# Generate and print classification metrics and confusion matrix

print(classification_report(y_true, y_pred, labels=CLASSES, target_names=CLASS_NAMES))
report = classification_report(y_true, y_pred, labels=CLASSES, target_names=CLASS_NAMES, output_dict=True)
with open(output_directory + 'classification_report.csv', 'w') as f:
    for key in report.keys():
        f.write("%s,%s\n" % (key, report[key]))
conf_arr = confusion_matrix(y_true, y_pred, labels=CLASSES)
print(conf_arr)
np.savetxt(output_directory + "confusion_matrix.csv", conf_arr, delimiter=",")

# Clear model from GPU after each iteration
print("Finished testing")


In [ ]:
!zip -r ./output.zip ./output

In [6]:
import numpy as np
import cv2
from keras.models import load_model
from tensorflow.keras.applications.vgg19 import preprocess_input

# Load the trained model
MODEL_PATH = "/kaggle/working/output/lastbest-46.keras"
model = load_model(MODEL_PATH)

# Define class labels
CLASS_NAMES = ['Chinee Apple', 'Lantana', 'Parkinsonia', 'Parthenium', 'Prickly Acacia', 
               'Rubber Vine', 'Siam Weed', 'Snake Weed', 'Negative']  

def predict_image(image_path):
    """Loads an image, preprocesses it, and predicts the weed type."""
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Resize to match model input size
    IMG_SIZE = (224, 224)  # Ensure this matches the training size
    image = cv2.resize(image, IMG_SIZE)
    
    # Convert to array and preprocess
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = preprocess_input(image)  # Normalize pixel values
    
    # Make prediction
    predictions = model.predict(image)
    
    # Get the highest probability class
    predicted_class = CLASS_NAMES[np.argmax(predictions)]
    confidence = np.max(predictions) * 100
    
    print(f"Predicted Class: {predicted_class} (Confidence: {confidence:.2f}%)")
    return predicted_class, confidence

# Example Usage:
image_path = "/kaggle/input/images/20160928-140314-0.jpg"  # Change this to your test image
predict_image(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 958ms/step
Predicted Class: Lantana (Confidence: 100.00%)


('Lantana', 99.99996423721313)